Going to perform a two-way repeated measures ANOVA. 

Load libraries

In [1]:
#library(xlsx)
library(rstatix)
#install.packages('ez')
library(ez)
#library(reshape)
#library(tidyverse)
#library(dplyr)
#library(ggpubr)
#library(plyr)
#library(datarium)

print("complete")


Attaching package: ‘rstatix’

The following object is masked from ‘package:stats’:

    filter

Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 


[1] "complete"


Load dataset

In [2]:
ldh_original <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/LDH_2wayANOVA.csv')
ldh_cleaned <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/LDH_2wayANOVA_cleaned.csv')
ldh_cleaned_cleaned <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/LDH_2wayANOVA_cleaned_cleaned.csv')

dd_original <- read.csv(file = "/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/DDimer_2wayANOVA.csv")
crp_original <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/Crp_2wayANOVA.csv')
b2m_original <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/b2m_2way_ANOVA.csv')


dataa <- b2m_original
dataa

study_id,treatment,visit_lab,score
1,1,1,3.480
1,1,2,5.570
1,1,3,4.860
2,1,1,1.830
2,1,2,2.010
2,1,3,1.900
3,2,2,1.850
3,2,3,1.970
4,1,1,31.090
5,1,1,3.060


Convert everything other than score into factors

In [3]:
convert_as_factor(dataa$study_id)
convert_as_factor(dataa$treatment)
convert_as_factor(dataa$visit_lab)

[1] 1      1      1      2      2      2      3      3      4      5     
[11] 5      5      6      6      6      7      7      8      8      8     
[21] 9      9      10     10     10     522-1  522-3  522-4  522-4  522-6 
[31] 522-7  522-7  522-8  522-8  522-9  522-10 523-1  523-2  523-2  524-1 
[41] 524-2  524-2  524-3  524-3  524-3  524-4  524-4  524-4  524-6  524-6 
[51] 524-6  524-7  524-7  524-7  524-8  524-9  524-11
29 Levels: 1 10 2 3 4 5 522-1 522-10 522-3 522-4 522-6 522-7 522-8 ... 9

[1] 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 1 1 1 1 1 2 2 2 2 1 1 1 2 2 2 1 1 2 2 1 2
[39] 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 2

[1] 1 2 3 1 2 3 2 3 1 1 2 3 1 2 3 1 2 1 2 3 1 2 1 2 3 1 1 1 3 1 1 2 1 2 1 1 1 1
[39] 2 1 1 2 1 2 3 1 2 3 1 2 3 1 2 3 1 1 1

Summary Statistics

In [4]:
summary<-dataa %>%
group_by(treatment,visit_lab) %>%
get_summary_stats(score, type = "mean_sd")
data.frame(summary)

treatment,visit_lab,variable,n,mean,sd
1,1,score,15,4.889,7.348
1,2,score,9,2.848,1.409
1,3,score,7,2.766,1.340
2,1,score,13,3.310,1.651
2,2,score,8,2.785,1.173
2,3,score,5,2.238,0.534


Outlier detection

In [5]:
outlier<-dataa %>%
group_by(treatment,visit_lab) %>%
identify_outliers(score)
data.frame(outlier)
print("Here are the outliers. Decide what to do with them later. Most likely will need to take them out in the future")


treatment,visit_lab,study_id,score,is.outlier,is.extreme
1,1,4,31.090,TRUE,TRUE
1,1,8,6.620,TRUE,FALSE
1,2,1,5.570,TRUE,FALSE
2,1,522-1,5.160,TRUE,FALSE
2,1,523-2,5.026,TRUE,FALSE
2,1,524-1,7.660,TRUE,TRUE
2,2,523-2,5.160,TRUE,TRUE


[1] "Here are the outliers. Decide what to do with them later. Most likely will need to take them out in the future"


Normality Checking

In [6]:
normality<-dataa %>%
group_by(treatment,visit_lab) %>%
shapiro_test(score)
data.frame(normality)
print("need p > 0.05 to satisfy normality assumption")

treatment,visit_lab,variable,statistic,p
1,1,score,0.4128639,7.532349e-07
1,2,score,0.7355952,3.688697e-03
1,3,score,0.8805757,2.290158e-01
2,1,score,0.7399459,1.435727e-03
2,2,score,0.8140017,4.034023e-02
2,3,score,0.9747512,9.047779e-01


[1] "need p > 0.05 to satisfy normality assumption"


Applying anova test with sphericity corrections 

In [7]:
#res.aov <- anova_test(
#data = dataa, dv = score, wid = study_id,
#within = c(treatment, visit_lab)
#)
#get_anova_table(res.aov)

Applying ANOVA with another package

In [8]:
newModel<-ezANOVA(
    data = dataa, dv = .(score), wid = .(study_id), 
    within = .(visit_lab), between = .(treatment), detailed = FALSE, type = 2)
newModel

Warning message:
“"visit_lab" will be treated as numeric.”Warning message:
“"treatment" will be treated as numeric.”Warning message:
“Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA().”

ERROR: Error in ezANOVA_main(data = data, dv = dv, wid = wid, within = within, : One or more cells is missing data. Try using ezDesign() to check your data.


Pairwise comparisons between treatment groups at each time point

In [ ]:
dataa %>%
pairwise_t_test(
score ~ treatment, paired = TRUE,
p.adjust.method = "bonferroni"
)

Comparisons for the time variable

In [ ]:
dataa %>%
pairwise_t_test(
score ~ visit_lab, paired = TRUE,
p.adjust.method = "bonferroni"
)